In [7]:
import numpy as np
import pandas as pd 
import matplotlib as plt
import sympy as sp

In [3]:
df = pd.read_csv("stabs.txt", header=None)
df

,0
0,1
1,1
2,0
3,0
4,1
5,0
6,1
7,0
8,1
9,0


In [6]:
df[0].to_list()

[1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]

In [12]:
s, k = sp.symbols('s k', rational=True)
K_a = 1/((s+1)*(s+2))
G_a = k

In [13]:
K_a

1/((s + 1)*(s + 2))

In [14]:
G_a

k

In [25]:
# Uklad zamkniety
K_a_z = K_a/(1+K_a*G_a)
K_a_z

1/((s + 1)*(s + 2)*(k/((s + 1)*(s + 2)) + 1))

In [26]:
# Uklad otwarty
K_a_0 = K_a*G_a
K_a_0

k/((s + 1)*(s + 2))

In [32]:
s = sp.Symbol('s', real=True)
a, b = Hurwitz_sp((1/((s+1)*(s+2)*(s+1))))


\left[\begin{matrix}\end{matrix}\right]


1

In [4]:
s = sp.Symbol('s', real=True)
a, b = Hurwitz_sp((1/((s+1)*(s+2)*(s+1))))


In [34]:
# Licznik i Mianownik K'atych (jako listy wspolczynnikow przy s'ach)
L_KG_LIST_a, M_KG_LIST_a = [[float(i) for i in sp.Poly(
    i, s).all_coeffs()] for i in (K_a*G_a).replace(k, 1).as_numer_denom()]
L_KG_LIST_b, M_KG_LIST_b = [[float(i) for i in sp.Poly(
    i, s).all_coeffs()] for i in (K_b*G_b).replace(k, 1).as_numer_denom()]
L_KG_LIST_c, M_KG_LIST_c = [[float(i) for i in sp.Poly(
    i, s).all_coeffs()] for i in (K_c*G_c).replace(k, 1).as_numer_denom()]
L_KG_LIST_d, M_KG_LIST_d = [[float(i) for i in sp.Poly(
    i, s).all_coeffs()] for i in (K_d*G_d).replace(k, 1).as_numer_denom()]

In [36]:
M_KG_LIST_c

[1.0, 4.0, 3.0]

In [6]:
s = sp.print_latex(K_a)

\frac{1}{\left(s + 1\right) \left(s + 2\right)}


In [11]:
a = str(sp.latex(K_a))

In [26]:
K_a+1

1 + 1/((s + 1)*(s + 2))

In [28]:
L_K_a

[1.0, 3.0, 3.0]

In [29]:
L_K_a, M_K_a = sp.fraction(K_a)
L_K_b, M_K_b = sp.fraction(K_b)
L_K_c, M_K_c = sp.fraction(K_c)
L_K_d, M_K_d = sp.fraction(K_d)

L_G_a, M_G_a = sp.fraction(G_a)
L_G_b, M_G_b = sp.fraction(G_b)
L_G_c, M_G_c = sp.fraction(G_c)
L_G_d, M_G_d = sp.fraction(G_d)

In [32]:
L_K_b

k

In [31]:
M_G_c

s + 3

In [33]:
L_KG_LIST_c

NameError: name 'L_KG_LIST_c' is not defined

In [37]:
df = pd.read_csv("stabs.txt", header=None)
# STABS_KG = df[0].to_list()
df

,0,1
0,1,1
1,1,1
2,1,1
3,1,1


In [38]:
STABS_KG = df[0].to_list()
STABS_KG

[1, 1, 1, 1]

In [39]:
df[1]

0    1
1    1
2    1
3    1
Name: 1, dtype: int64

In [1]:
'''
W tym pliku zdefiniowane sa funkcje potrzebne do kryterium Hurwitza
'''
import sympy as sp
import numpy as np
sp.init_printing(use_latex=True)
sp.init_printing(use_unicode=True)


def Hurwitz_sp(Y):
    '''
    Funkcja sprawdzajaca stabilnosc transmitancji
    INPUT: obiekt sp.Function
    OUTPUT: stabilnosc 1 lub 0, macierz Hurwitza transmitancji(tex syndax)
    '''
    # zainicjowanie symboli
    s = sp.Symbol('s', real=True)
    M = sp.Function('M')(s)
    L = sp.Function('L')(s)
    L, M = sp.fraction(Y)  # podział na licznik i mianownik
    Ywspolczynniki = sp.Poly(M, s)
    Ywspolczynniki = Ywspolczynniki.all_coeffs()
    # rozpoczynam algorytm kryterium Hurwitza
    Ystopien = len(Ywspolczynniki)-1
    # tworze wektory potrzebne do stworzenia macierzy Hurwitza
    pTabParz = sp.Matrix([[]])
    pTabNParz = sp.Matrix([[]])
    pm = 0
    pn = 0
    for x in range(Ystopien):
        pm = x*2
        pn = x*2+1
        if pm <= Ystopien:
            pTabParz = pTabParz.col_insert(x, sp.Matrix([Ywspolczynniki[pm]]))
        else:
            pTabParz = pTabParz.col_insert(x, sp.Matrix([0]))
        if pn <= Ystopien:
            pTabNParz = pTabNParz.col_insert(
                x, sp.Matrix([Ywspolczynniki[pn]]))
        else:
            pTabNParz = pTabNParz.col_insert(x, sp.Matrix([0]))
    # deklaruje macierz Hurwitza jako macierz zer
    macierzHurwitza = sp.zeros(Ystopien, Ystopien)
    # algorytm zapusijacy macierz Hurwitza
    parzyste = False
    if Ystopien == 0:
        macierzHurwitza = (sp.ones(1, 1))/Y
    else:
        for x in range(Ystopien):
            if(parzyste == False):
                macierzHurwitza.row_del(x)
                macierzHurwitza = macierzHurwitza.row_insert(x, pTabNParz)
                pTabNParz = pTabNParz.col_insert(0, sp.Matrix([0]))
                pTabNParz.col_del(Ystopien)
                parzyste = (not parzyste)
            elif(parzyste == True):
                macierzHurwitza.row_del(x)
                macierzHurwitza = macierzHurwitza.row_insert(x, pTabParz)
                pTabParz = pTabParz.col_insert(0, sp.Matrix([0]))
                pTabParz.col_del(Ystopien)
                parzyste = (not parzyste)
    # algorytm sprawdzajacy czy wszystkie podwyznaczniki sa dodatnie
    stabilnoscHurwitza = 1
    macierzHurwitzaPomocnicza = macierzHurwitza.copy()
    wyznacznikiHurwitza=[]
    for x in range(Ystopien):
        wyznacznikiHurwitza.append(macierzHurwitzaPomocnicza.det().copy())
        if macierzHurwitzaPomocnicza.det() <= 0:
            stabilnoscHurwitza = 0
        macierzHurwitzaPomocnicza.col_del(macierzHurwitzaPomocnicza.shape[0]-1)
        macierzHurwitzaPomocnicza.row_del(macierzHurwitzaPomocnicza.shape[0]-1)
        # jesli wszystkie podwyznaczniki sa dodatnie to zmienna stabilnoscHurwitza jest rowna 1, w przeciwnym wypadku jest rowna 0
    return stabilnoscHurwitza, str(sp.latex(macierzHurwitza)), wyznacznikiHurwitza


In [2]:
# Ustawiamy s oraz k w SymPy
s = sp.Symbol('s', rational=True)
k = sp.Symbol('k', rational=True, real=True)

# Nasze transmitancje i ich postacie w latexu
K_a = 1/((s+1)*(s+2))
K_b = k
K_c = (s+2)/((s+1)*(s+2))
K_d = 1/(s**2+4*s+5)

LATEX_K_a = str(sp.latex(K_a))
LATEX_K_b = str(sp.latex(K_b))
LATEX_K_c = str(sp.latex(K_c))
LATEX_K_d = str(sp.latex(K_d))

G_a = k
G_b = 1/((s+1)*(s+2))
G_c = 1/(s+3)
G_d = 1/(s+3)


LATEX_G_a = str(sp.latex(G_a))
LATEX_G_b = str(sp.latex(G_b))
LATEX_G_c = str(sp.latex(G_c))
LATEX_G_d = str(sp.latex(G_d))


# Podział na licznik i mianownik (symbolicznie)
L_K_a, M_K_a = sp.fraction(K_a)
L_K_b, M_K_b = sp.fraction(K_b)
L_K_c, M_K_c = sp.fraction(K_c)
L_K_d, M_K_d = sp.fraction(K_d)

L_G_a, M_G_a = sp.fraction(G_a)
L_G_b, M_G_b = sp.fraction(G_b)
L_G_c, M_G_c = sp.fraction(G_c)
L_G_d, M_G_d = sp.fraction(G_d)

# Hurwitz, dla kazdego przykladu, dla petli zamknietej
# Dostajemy booleana czy jest stabilny czy nie
# oraz wersje latexowa(string) K_z
K_Z_STAB_a, LATEX_K_Z_a, K_Z_DET_a = Hurwitz_sp(L_K_a*L_G_a+M_K_a*M_G_a)
K_Z_STAB_b, LATEX_K_Z_b, K_Z_DET_b = Hurwitz_sp(L_K_b*L_G_b+M_K_b*M_G_b)
K_Z_STAB_c, LATEX_K_Z_c, K_Z_DET_c = Hurwitz_sp(L_K_c*L_G_c+M_K_c*M_G_c)
K_Z_STAB_d, LATEX_K_Z_d, K_Z_DET_d = Hurwitz_sp(L_K_d*L_G_d+M_K_d*M_G_d)

In [3]:
K_Z_STAB_d, LATEX_K_Z_d, K_Z_DET_d

(1,
 '\\left[\\begin{matrix}\\frac{1}{\\left(s + 3\\right) \\left(s^{2} + 4 s + 5\\right) + 1}\\end{matrix}\\right]',
 [])

In [ ]:
K_Z_STAB_d
